In [1]:
import pandas as pd
import numpy as np

In [8]:
xls = pd.ExcelFile(r'C:\Users\owner\Google Drive 2\MSc\TU DELFT - Water Managment\Additional Thesis\Results\koppen_valid_stations.xlsx')
df = pd.read_excel(xls, 'all')
df

,Unique Code,Station,P1,P2,P3,P4,P5,P6,P7,P8,...,T4,T5,T6,T7,T8,T9,T10,T11,T12,Z
0,eu-1,32,58.100000,53.900000,53.700000,53.700000,75.800000,59.000000,54.400000,58.300000,...,10.500000,14.100000,17.400000,19.700000,19.400000,16.500000,12.300000,7.200000,4.400000,161.0000
1,eu-2,34,86.100000,72.400000,66.100000,65.500000,66.800000,61.100000,48.000000,56.600000,...,11.800000,15.200000,18.400000,20.400000,20.400000,18.000000,14.100000,9.300000,6.600000,47.0000
2,eu-3,36,49.600000,45.800000,54.100000,52.600000,48.200000,29.500000,18.400000,29.200000,...,13.800000,17.200000,21.200000,24.000000,23.700000,20.700000,16.500000,11.800000,9.000000,42.0000
3,eu-4,39,44.700000,37.900000,39.600000,45.000000,40.800000,25.500000,11.900000,27.600000,...,13.300000,17.200000,21.100000,23.800000,23.400000,20.300000,16.000000,10.900000,7.400000,9.0000
4,eu-5,40,39.000000,31.400000,35.900000,35.100000,53.100000,59.500000,59.900000,53.100000,...,8.700000,13.300000,16.700000,18.400000,17.600000,13.600000,8.800000,4.000000,1.000000,240.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Ca-56,161,41.988235,46.494118,31.800000,23.047059,9.064706,3.417647,0.882353,0.066667,...,15.088235,19.294118,22.647059,24.058824,23.016667,21.611111,17.429412,11.447059,7.700000,55.7784
521,Ca-57,174,19.632667,58.233333,31.302000,14.957500,8.141875,0.120000,2.723750,0.450000,...,14.481250,16.192857,18.100000,20.843750,20.993750,20.723529,18.037500,14.450000,11.653333,5.1816
522,Ca-58,173,27.853333,40.350000,21.946667,18.237500,9.337500,2.093750,5.006250,2.581250,...,13.125000,14.831250,16.081250,18.193750,18.687500,18.525000,17.153333,15.306250,12.875000,102.1080
523,Ca-59,150,41.231250,60.406250,27.470588,19.583333,5.983333,0.322222,0.155556,0.427778,...,14.672222,16.377778,17.972222,20.005556,20.555556,20.133333,18.011765,14.843750,12.225000,135.6360


In [9]:
def Koppen(P, T, z = 0):
    if np.isnan(P).all() == True:
        classification = 'Not classified'
    else:
        data = pd.DataFrame(data = (P, T), index = ['P', 'T']).transpose()

        MAP = np.sum(P)
        MAT = np.mean(T)
        Thot = np.max(T)
        Tcold = np.min(T)
        Tmonth10 = sum(map(lambda x : x > 10, T))
        Pdry = np.min(P)
        seqT = np.array((T[0], T[1], T[2], T[3], T[4], T[5], T[6], T[7], T[8], T[9], T[10], T[11], T[0], T[1], T[2], T[3], T[4]))
        seqP = np.array((P[0], P[1], P[2], P[3], P[4], P[5], P[6], P[7], P[8], P[9], P[10], P[11], P[0], P[1], P[2], P[3], P[4]))
        data['seqT'] = np.convolve(seqT, np.ones(6, dtype=np.int), mode='valid')
        data['seqP'] = np.convolve(seqP, np.ones(6, dtype=np.int), mode='valid')

        s_st = data['seqT'].idxmax(axis = 1)
        s_f = s_st + 6
        w_st = s_f
        w_f = w_st + 6

        if s_f > 12:
            s_f = s_f - 12
        if w_f > 12:
            w_f = w_f - 12
        if s_st > 12:
            s_st = s_st - 12
        if w_st > 12:
            w_st = w_st - 12

        drymonth = data['P'].idxmin(axis = 1)
        season = []

        if s_st < s_f and w_st < w_f:
            Psdry = data['P'][(data.index >= s_st) & (data.index < s_f)].min()
            Pwdry = (data['P'][(data.index >= w_st) & (data.index < w_f)].min())
            Pswet = data['P'][(data.index >= s_st) & (data.index < s_f)].max()
            Pwwet = (data['P'][(data.index >= w_st) & (data.index < w_f)].max())
            if s_st < drymonth and s_f > drymonth:
                season.append('summer')
            else:
                season.append('winter')

        if s_st < s_f and w_st > w_f:
            Psdry = data['P'][(data.index >= s_st) & (data.index < s_f)].min()
            Pwdry = (data['P'][(data.index >= w_st)].min(), data['P'][(data.index < w_f)].min())
            Pwdry = np.min(Pwdry)
            Pswet = data['P'][(data.index >= s_st) & (data.index < s_f)].max()
            Pwwet = (data['P'][(data.index >= w_st)].max(), data['P'][(data.index < w_f)].max())
            Pwwet = np.max(Pwwet)
            if s_st < drymonth and s_f > drymonth:
                season.append('summer')
            else:
                season.append('winter')

        if s_st > s_f and w_st < w_f:
            Psdry = (data['P'][(data.index >= s_st)].min(), data['P'][(data.index < s_f)].min())
            Psdry = np.min(Psdry)
            Pwdry = (data['P'][(data.index >= w_st) & (data.index < w_f)].min())
            Pswet = (data['P'][(data.index >= s_st)].max(), data['P'][(data.index < s_f)].max())
            Pswet = np.max(Pswet)
            Pwwet = (data['P'][(data.index >= w_st) & (data.index < w_f)].max())

            if w_st < drymonth and w_f > drymonth:
                season.append('winter')
            else:
                season.append('summer')

        if s_st < w_st:
            s_f = s_f - 1
            w_f = w_f - 1
        else:
            w_f = w_f - 1
            s_f = s_f - 1

        if w_st == 12:
            w_st = 0

        if 0.7 * MAP <= data['seqP'].loc[w_st]:
            Pthresh = 2 * MAT
        elif 0.7 * MAP <= data['seqP'].loc[s_st]:
            Pthresh = 2 * MAT + 28
        else:
            Pthresh = 2 * MAT + 14

        class_Kop = []

        if z >= 2300:
            if Thot >= 0:
                class_Kop.append('HT')
            else:
                class_Kop.append('HF')
        else:
            if Thot <= 10:
                if Thot > 0:
                    class_Kop.append('ET')
                else:
                    class_Kop.append('EF')
            else:
                if MAP < 10 * Pthresh:
                    if MAP < 5 * Pthresh:
                        if MAT >= 18:
                             class_Kop.append('BWh')
                        else:
                             class_Kop.append('BWk')
                    else:
                        if MAT >= 18:
                             class_Kop.append('BSh')
                        else:
                             class_Kop.append('BSk')
                else:
                    if Tcold >= 18:
                        if Pdry > 60:
                            class_Kop.append('Af')
                        else:
                            if MAP < (100 - (Pdry) * 25):
                                class_Kop.append('Am')
                            else:
                                if season == 'summer':
                                    class_Kop.append('As')
                                else:
                                    class_Kop.append('Aw')
                    else:
                        if Tcold > -3:
                            if Pwdry > Psdry and Pwdry > Psdry * 3 and Psdry < 40:
                                class_Kop.append('Cs')
                                if Thot >= 22:
                                    class_Kop.append('a')
                                elif Thot < 22 and Tmonth10 >= 4:
                                    class_Kop.append('b')
                                else:
                                    class_Kop.append('c') 
                            elif Psdry > Pwdry and Pswet > Pwdry * 10:
                                class_Kop.append('Cw')
                                if Thot >= 22:
                                    class_Kop.append('a')
                                elif Thot < 22 and Tmonth10 >= 4:
                                    class_Kop.append('b')
                                else:
                                    class_Kop.append('c')
                            else:
                                class_Kop.append('Cf')
                                if Thot >= 22:
                                    class_Kop.append('a')
                                elif Thot < 22 and Tmonth10 >= 4:
                                    class_Kop.append('b')
                                else:
                                    class_Kop.append('c')
                        else:
                            if Pwdry > Psdry and Pwdry > Psdry * 3 and Psdry < 40:
                                class_Kop.append('Ds')
                                if Thot >= 22:
                                    class_Kop.append('a')
                                elif Thot < 22 and Tmonth10 >= 4:
                                    class_Kop.append('b')
                                elif Tmonth10 < 4 and Tcold >= -38:
                                    class_Kop.append('c')
                                else:
                                    class_Kop.append('d')
                            elif Psdry > Pwdry and Pswet > Pwdry * 10:
                                class_Kop.append('Dw')
                                if Thot >= 22:
                                    class_Kop.append('a')
                                elif Thot < 22 and Tmonth10 >= 4:
                                    class_Kop.append('b')
                                elif Tmonth10 < 4 and Tcold >= -38:
                                    class_Kop.append('c')
                                else:
                                    class_Kop.append('d')
                            else:
                                class_Kop.append('Df')
                                if Thot >= 22:
                                    class_Kop.append('a')
                                elif Thot < 22 and Tmonth10 >= 4:
                                    class_Kop.append('b')
                                elif Tmonth10 < 4 and Tcold >= -38:
                                    class_Kop.append('c')
                                else:
                                    class_Kop.append('d')
        classification = ''.join(class_Kop)
    return classification, MAP, MAT, Thot, Tcold, Tmonth10, Pdry, Psdry, Pwdry, Pswet, Pwwet, Pthresh, season, data

In [10]:
T = (df['T1'], df['T2'], df['T3'], df['T4'], df['T5'], df['T6'], df['T7'], df['T8'], df['T9'], df['T10'], df['T11'], df['T12'])
P = (df['P1'], df['P2'], df['P3'], df['P4'], df['P5'], df['P6'], df['P7'], df['P8'], df['P9'], df['P10'], df['P11'], df['P12'])
z = df['Z']

In [11]:
world_kop = []
for i in df.index:
    world_kop.append(Koppen((df['P1'][i], df['P2'][i], df['P3'][i], df['P4'][i], df['P5'][i], df['P6'][i], df['P7'][i], df['P8'][i], df['P9'][i], df['P10'][i], df['P11'][i], df['P12'][i]), (df['T1'][i], df['T2'][i], df['T3'][i], df['T4'][i], df['T5'][i], df['T6'][i], df['T7'][i], df['T8'][i], df['T9'][i], df['T10'][i], df['T11'][i], df['T12'][i]), 0)[0])

df['classification'] = np.array(world_kop)

In [12]:
df.to_excel("validation_stations_koppen.xlsx")

In [11]:
df

,Unique Code,Station,P1,P2,P3,P4,P5,P6,P7,P8,...,T5,T6,T7,T8,T9,T10,T11,T12,Z,classification
0,eu-1,32,58.100000,53.900000,53.700000,53.700000,75.800000,59.000000,54.400000,58.300000,...,14.100000,17.400000,19.700000,19.400000,16.500000,12.300000,7.200000,4.400000,161.0000,Cfb
1,eu-2,34,86.100000,72.400000,66.100000,65.500000,66.800000,61.100000,48.000000,56.600000,...,15.200000,18.400000,20.400000,20.400000,18.000000,14.100000,9.300000,6.600000,47.0000,Cfb
2,eu-3,36,49.600000,45.800000,54.100000,52.600000,48.200000,29.500000,18.400000,29.200000,...,17.200000,21.200000,24.000000,23.700000,20.700000,16.500000,11.800000,9.000000,42.0000,Cfa
3,eu-4,39,44.700000,37.900000,39.600000,45.000000,40.800000,25.500000,11.900000,27.600000,...,17.200000,21.100000,23.800000,23.400000,20.300000,16.000000,10.900000,7.400000,9.0000,Csa
4,eu-5,40,39.000000,31.400000,35.900000,35.100000,53.100000,59.500000,59.900000,53.100000,...,13.300000,16.700000,18.400000,17.600000,13.600000,8.800000,4.000000,1.000000,240.0000,Cfb
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,Ca-56,161,41.988235,46.494118,31.800000,23.047059,9.064706,3.417647,0.882353,0.066667,...,19.294118,22.647059,24.058824,23.016667,21.611111,17.429412,11.447059,7.700000,55.7784,BSk
521,Ca-57,174,19.632667,58.233333,31.302000,14.957500,8.141875,0.120000,2.723750,0.450000,...,16.192857,18.100000,20.843750,20.993750,20.723529,18.037500,14.450000,11.653333,5.1816,BSk
522,Ca-58,173,27.853333,40.350000,21.946667,18.237500,9.337500,2.093750,5.006250,2.581250,...,14.831250,16.081250,18.193750,18.687500,18.525000,17.153333,15.306250,12.875000,102.1080,BSk
523,Ca-59,150,41.231250,60.406250,27.470588,19.583333,5.983333,0.322222,0.155556,0.427778,...,16.377778,17.972222,20.005556,20.555556,20.133333,18.011765,14.843750,12.225000,135.6360,BSk
